In [4]:
import numpy as np
import pandas as pd
import sys
from hazm import *
from stopwords_guilannlp import *
from nltk.tokenize import RegexpTokenizer
from string import punctuation
import re
import math
import xlrd
import nltk
import math
normalizer = Normalizer()
lemmatizer = Lemmatizer()
stemmer = Stemmer()


In [7]:
dataset=pd.read_csv("ProjectData.csv")

In [8]:
dataset=dataset[['comment','label']]

In [9]:
dataset=dataset[dataset['label'] != -2]

In [10]:
dataset=dataset.dropna()


In [11]:
def undersampl_data(dataset):
    counts = dataset['label'].value_counts()
    min_count = counts.min()
    undersampled_df = pd.concat([
        dataset[dataset['label'] == Label].sample(n=min_count)
        for Label in counts.index
    ])
    
    return undersampled_df

In [12]:
dataset=undersampl_data(dataset)

In [13]:
counts = dataset ['label'].value_counts()

In [14]:
counts

 1.0    5627
-1.0    5627
 0.0    5627
Name: label, dtype: int64

In [16]:
dataset

,comment,label
10425,من وقتی بدستم رسید وشارژ اولیه رو زدم تا 35 رو...,1.0
26499,خیلی بامزه هست...دقیقا همونی هست که تو تصویره....,1.0
1272,دستبند خیلی خیلی قشنگیه مخصوصا نگین های مختلفش...,1.0
7382,کیفیتش عالی هست و اصلا فکر نمیکنید که برچسب زد...,1.0
10954,واقعا گاز خوبیه راضی ام‌ازش,1.0
...,...,...
3964,به قیمتش اصلا ارزنده نیست,0.0
37192,من تقریبا ۱ سال شده که این موس رو دارم و هیچ م...,0.0
5081,کفپوش قشنگیه ولی میتونست برای قسمت جلوی ماشین ...,0.0
46915,سلام دوستانعزیزانی که از لق شدن روپدالی شکایت ...,0.0


In [12]:
def remove_stop_words(comments):
    stop_words = ['و', 'در', 'به', 'از', 'که', 'این', 'را', 'با', 'است', 'برای', 'آن', 'یک', 'خود', 'تا', 'کرد', 'بر', 'هم', 'نیز', 'گفت', 'می\u200cشود', 'وی', 'شد', 'دارد', 'ما', 'اما', 'یا', 'شده', 'باید', 'هر', 'آنها', 'بود', 'او', 'دیگر', 'دو', 'مورد', 'می\u200cکند', 'شود', 'کند', 'وجود', 'بین', 'پیش', 'شده_است', 'پس', 'نظر', 'اگر', 'همه', 'یکی', 'حال', 'هستند', 'من', 'کنند', 'نیست', 'باشد', 'چه', 'بی', 'می', 'بخش', 'می\u200cکنند', 'همین', 'افزود', 'هایی', 'دارند', 'راه', 'همچنین', 'روی', 'داد', 'سه', 'داشت', 'چند', 'سوی', 'تنها', 'هیچ', 'میان', 'اینکه', 'شدن', 'بعد', 'جدید', 'ولی', 'حتی', 'کردن', 'برخی', 'کردند', 'می\u200cدهد', 'اول', 'نه', 'کرده_است', 'نسبت', 'بیش', 'شما', 'چنین', 'طور', 'افراد', 'تمام', 'درباره', 'بار', 'بسیاری', 'می\u200cتواند', 'کرده', 'چون', 'ندارد', 'دوم', 'بزرگ', 'طی', 'حدود', 'همان', 'بدون', 'البته', 'آنان', 'می\u200cگوید', 'دیگری', 'خواهد_شد', 'کنیم', 'قابل', 'یعنی', 'رشد', 'می\u200cتوان', 'وارد', 'کل', 'ویژه', 'قبل', 'براساس', 'نیاز', 'گذاری', 'هنوز', 'لازم', 'سازی', 'بوده_است', 'چرا', 'می\u200cشوند', 'وقتی', 'گرفت', 'کم', 'جای', 'حالی', 'تغییر', 'پیدا', 'اکنون', 'تحت', 'باعث', 'مدت', 'فقط', 'تعداد', 'آیا', 'بیان', 'رو', 'شدند', 'عدم', 'کرده_اند', 'بودن', 'نوع', 'بلکه', 'جاری', 'دهد', 'برابر', 'مهم', 'بوده', 'اخیر', 'مربوط', 'امر', 'زیر', 'گیری', 'شاید', 'خصوص', 'آقای', 'اثر', 'کننده', 'بودند', 'فکر', 'کنار', 'اولین', 'سوم', 'سایر', 'کنید', 'ضمن', 'مانند', 'باز', 'می\u200cگیرد', 'ممکن', 'حل', 'دارای', 'پی', 'مثل', 'می\u200cرسد', 'اجرا', 'دور', 'منظور', 'کسی', 'موجب', 'طول', 'امکان', 'آنچه', 'تعیین', 'گفته', 'شوند', 'جمع', 'علاوه', 'گونه', 'تاکنون', 'رسید', 'ساله', 'گرفته', 'شده_اند', 'علت', 'چهار', 'داشته_باشد', 'خواهد_بود', 'طرف', 'تهیه', 'تبدیل', 'مناسب', 'زیرا', 'مشخص', 'می\u200cتوانند', 'نزدیک', 'جریان', 'روند', 'بنابراین', 'می\u200cدهند', 'یافت', 'نخستین', 'بالا', 'پنج', 'ریزی', 'چیزی', 'نخست', 'بیشتری', 'ترتیب', 'شده_بود', 'خاص', 'شروع', 'فرد', 'کامل', 'غیر', 'می\u200cرود', 'دهند', 'آخرین', 'دادن', 'جدی', 'بهترین', 'شامل', 'گیرد', 'بخشی', 'باشند', 'تمامی', 'بهتر', 'داده_است', 'حد', 'نبود', 'کسانی', 'می\u200cکرد', 'داریم', 'علیه', 'می\u200cباشد', 'دانست', 'ناشی', 'داشتند', 'دهه', 'می\u200cشد', 'ایشان', 'آنجا', 'گرفته_است', 'دچار', 'می\u200cآید', 'لحاظ', 'آنکه', 'داده', 'بعضی', 'هستیم', 'اند', 'برداری', 'نباید', 'می\u200cکنیم', 'نشست', 'سهم', 'همیشه', 'آمد', 'اش', 'وگو', 'می\u200cکنم', 'حداقل', 'طبق', 'جا', 'خواهد_کرد', 'نوعی', 'چگونه', 'رفت', 'هنگام', 'فوق', 'روش', 'ندارند', 'سعی', 'بندی', 'شمار', 'کلی', 'کافی', 'مواجه', 'همچنان', 'سمت', 'کوچک', 'داشته_است', 'چیز', 'پشت', 'آورد', 'حالا', 'روبه', 'سال\u200cهای', 'دادند', 'می\u200cکردند', 'عهده', 'نیمه', 'جایی', 'دیگران', 'سی', 'بروز', 'یکدیگر', 'آمده_است', 'جز', 'کنم', 'سپس', 'کنندگان', 'خودش', 'همواره', 'یافته', 'شان', 'صرف', 'نمی\u200cشود', 'رسیدن', 'چهارم', 'یابد', 'متر', 'ساز', 'داشته', 'کرده_بود', 'باره', 'نحوه', 'کردم', 'تو', 'شخصی', 'داشته_باشند', 'محسوب', 'پخش', 'کمی', 'متفاوت', 'سراسر', 'کاملا', 'داشتن', 'نظیر', 'آمده', 'گروهی', 'فردی', 'ع', 'همچون', 'خطر', 'خویش', 'کدام', 'دسته', 'سبب', 'عین', 'آوری', 'متاسفانه', 'بیرون', 'دار', 'ابتدا', 'شش', 'افرادی', 'می\u200cگویند', 'سالهای', 'درون', 'نیستند', 'یافته_است', 'پر', 'خاطرنشان', 'گاه', 'جمعی', 'اغلب', 'دوباره', 'می\u200cیابد', 'لذا', 'زاده', 'گردد', 'اینجا']
    REPLACE_NO_SPACE = re.compile("[.`;:!\'?,\"()\[\]،؛ًٌٍَُِّ]")
    REPLACE_NUMBER_ENGLISH = re.compile("[0-9A_Za-z۰-۹]")
    clean_comments = []
    comments = [REPLACE_NO_SPACE.sub("", line) for line in comments]
    comments = [REPLACE_NUMBER_ENGLISH.sub("", line) for line in comments]
    for review in comments:
        clean_comments.append(
            ' '.join([word for word in review.split() 
                      if word not in stop_words])
        )
    return clean_comments

In [13]:
def tokenize_text(text):
    text = normalizer.normalize(text)
    text = text.replace('.', ' ')
    text = re.sub('\s+', ' ', text).strip()
    text = text.replace('\u200c', ' ').replace('\n', '').replace('\r', '').replace('ي', 'ی').replace('ك', 'ک')
    tokens = word_tokenize(text)
    return tokens


In [14]:
def create_word_set(comments):
    word_set = set()
    for comment in comments:
        for token in comment:
            word_set.add(token)
    return word_set

In [17]:
def preprocessing(comment):
    stop_words = ['و', 'در', 'به', 'از', 'که', 'این', 'را', 'با', 'است', 'برای', 'آن', 'یک', 'خود', 'تا', 'کرد', 'بر', 'هم', 'نیز', 'گفت', 'می\u200cشود', 'وی', 'شد', 'دارد', 'ما', 'اما', 'یا', 'شده', 'باید', 'هر', 'آنها', 'بود', 'او', 'دیگر', 'دو', 'مورد', 'می\u200cکند', 'شود', 'کند', 'وجود', 'بین', 'پیش', 'شده_است', 'پس', 'نظر', 'اگر', 'همه', 'یکی', 'حال', 'هستند', 'من', 'کنند', 'نیست', 'باشد', 'چه', 'بی', 'می', 'بخش', 'می\u200cکنند', 'همین', 'افزود', 'هایی', 'دارند', 'راه', 'همچنین', 'روی', 'داد', 'سه', 'داشت', 'چند', 'سوی', 'تنها', 'هیچ', 'میان', 'اینکه', 'شدن', 'بعد', 'جدید', 'ولی', 'حتی', 'کردن', 'برخی', 'کردند', 'می\u200cدهد', 'اول', 'نه', 'کرده_است', 'نسبت', 'بیش', 'شما', 'چنین', 'طور', 'افراد', 'تمام', 'درباره', 'بار', 'بسیاری', 'می\u200cتواند', 'کرده', 'چون', 'ندارد', 'دوم', 'بزرگ', 'طی', 'حدود', 'همان', 'بدون', 'البته', 'آنان', 'می\u200cگوید', 'دیگری', 'خواهد_شد', 'کنیم', 'قابل', 'یعنی', 'رشد', 'می\u200cتوان', 'وارد', 'کل', 'ویژه', 'قبل', 'براساس', 'نیاز', 'گذاری', 'هنوز', 'لازم', 'سازی', 'بوده_است', 'چرا', 'می\u200cشوند', 'وقتی', 'گرفت', 'کم', 'جای', 'حالی', 'تغییر', 'پیدا', 'اکنون', 'تحت', 'باعث', 'مدت', 'فقط', 'تعداد', 'آیا', 'بیان', 'رو', 'شدند', 'عدم', 'کرده_اند', 'بودن', 'نوع', 'بلکه', 'جاری', 'دهد', 'برابر', 'مهم', 'بوده', 'اخیر', 'مربوط', 'امر', 'زیر', 'گیری', 'شاید', 'خصوص', 'آقای', 'اثر', 'کننده', 'بودند', 'فکر', 'کنار', 'اولین', 'سوم', 'سایر', 'کنید', 'ضمن', 'مانند', 'باز', 'می\u200cگیرد', 'ممکن', 'حل', 'دارای', 'پی', 'مثل', 'می\u200cرسد', 'اجرا', 'دور', 'منظور', 'کسی', 'موجب', 'طول', 'امکان', 'آنچه', 'تعیین', 'گفته', 'شوند', 'جمع', 'علاوه', 'گونه', 'تاکنون', 'رسید', 'ساله', 'گرفته', 'شده_اند', 'علت', 'چهار', 'داشته_باشد', 'خواهد_بود', 'طرف', 'تهیه', 'تبدیل', 'مناسب', 'زیرا', 'مشخص', 'می\u200cتوانند', 'نزدیک', 'جریان', 'روند', 'بنابراین', 'می\u200cدهند', 'یافت', 'نخستین', 'بالا', 'پنج', 'ریزی', 'چیزی', 'نخست', 'بیشتری', 'ترتیب', 'شده_بود', 'خاص', 'شروع', 'فرد', 'کامل', 'غیر', 'می\u200cرود', 'دهند', 'آخرین', 'دادن', 'جدی', 'بهترین', 'شامل', 'گیرد', 'بخشی', 'باشند', 'تمامی', 'بهتر', 'داده_است', 'حد', 'نبود', 'کسانی', 'می\u200cکرد', 'داریم', 'علیه', 'می\u200cباشد', 'دانست', 'ناشی', 'داشتند', 'دهه', 'می\u200cشد', 'ایشان', 'آنجا', 'گرفته_است', 'دچار', 'می\u200cآید', 'لحاظ', 'آنکه', 'داده', 'بعضی', 'هستیم', 'اند', 'برداری', 'نباید', 'می\u200cکنیم', 'نشست', 'سهم', 'همیشه', 'آمد', 'اش', 'وگو', 'می\u200cکنم', 'حداقل', 'طبق', 'جا', 'خواهد_کرد', 'نوعی', 'چگونه', 'رفت', 'هنگام', 'فوق', 'روش', 'ندارند', 'سعی', 'بندی', 'شمار', 'کلی', 'کافی', 'مواجه', 'همچنان', 'سمت', 'کوچک', 'داشته_است', 'چیز', 'پشت', 'آورد', 'حالا', 'روبه', 'سال\u200cهای', 'دادند', 'می\u200cکردند', 'عهده', 'نیمه', 'جایی', 'دیگران', 'سی', 'بروز', 'یکدیگر', 'آمده_است', 'جز', 'کنم', 'سپس', 'کنندگان', 'خودش', 'همواره', 'یافته', 'شان', 'صرف', 'نمی\u200cشود', 'رسیدن', 'چهارم', 'یابد', 'متر', 'ساز', 'داشته', 'کرده_بود', 'باره', 'نحوه', 'کردم', 'تو', 'شخصی', 'داشته_باشند', 'محسوب', 'پخش', 'کمی', 'متفاوت', 'سراسر', 'کاملا', 'داشتن', 'نظیر', 'آمده', 'گروهی', 'فردی', 'ع', 'همچون', 'خطر', 'خویش', 'کدام', 'دسته', 'سبب', 'عین', 'آوری', 'متاسفانه', 'بیرون', 'دار', 'ابتدا', 'شش', 'افرادی', 'می\u200cگویند', 'سالهای', 'درون', 'نیستند', 'یافته_است', 'پر', 'خاطرنشان', 'گاه', 'جمعی', 'اغلب', 'دوباره', 'می\u200cیابد', 'لذا', 'زاده', 'گردد', 'اینجا']
    REPLACE_NO_SPACE = re.compile("[.`;:!\'?,\"()\[\]،؛ًٌٍَُِّ]")
    REPLACE_NUMBER_ENGLISH = re.compile("[0-9A_Za-z۰-۹]")
    comment = [REPLACE_NO_SPACE.sub("", line) for line in comment]
    comment = [REPLACE_NUMBER_ENGLISH.sub("", line) for line in comment]
    comment = ''.join(c for c in comment if not c.isdigit())
    comment = ''.join(c for c in comment if c not in punctuation)
    comment = normalizer.normalize(comment)
    tokens = word_tokenize(comment)
    cleared_text = []
    for word in tokens:
        word = normalizer.normalize(word)
#         word = stemmer.stem(word)
        word = lemmatizer.lemmatize(word)
          #word = re.sub(r'\d+', '', word)
        if word not in stop_words and len(word) > 1:
                cleared_text.append(word)
    return cleared_text

In [19]:
X=(dataset.iloc[:, :-1].values)
y=dataset.iloc[:, -1]

In [20]:
tokens=[]
for i in range(len(X)):
    tokens.append(preprocessing((X[i])))

In [22]:
len(tokens[0])

33

In [23]:
tf = []
for doc in tokens:
    doc_tf = {}
    for word in doc:
        doc_tf[word] = doc.count(word) / len(doc)
    tf.append(doc_tf)

# Calculate the inverse document frequency (IDF) for each word
idf = {}
for doc in tokens:
    for word in set(doc):
        if word in idf:
            idf[word] += 1
        else:
            idf[word] = 1

num_docs = len(tokens)
for word in idf:
    idf[word] = math.log((1 + num_docs) / (1 + idf[word])) + 1

# Multiply the TF and IDF to get the TF-IDF score for each word
tfidf = []
for doc in tf:
    doc_tfidf = {}
    for word in doc:
        doc_tfidf[word] = doc[word] * idf[word]
    tfidf.append(doc_tfidf)

# Normalize the TF-IDF score for each word in each document
for i in range(len(tfidf)):
    tfidf_values = list(tfidf[i].values())
    norm = math.sqrt(sum(x**2 for x in tfidf_values))
    for word in tfidf[i]:
        tfidf[i][word] /= norm

# Create a feature matrix
vocab = sorted(set(word for doc in tokens for word in doc))
matrix = [[doc.get(word, 0) for word in vocab] for doc in tfidf]
matrix = np.array(matrix)

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(matrix,y, test_size=0.2, random_state=42)


In [25]:

# Build the logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = model.predict(X_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)




C:\Users\ASUS\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.6538347645839503


In [26]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, y_train)

# Test the classifier
accuracy = clf.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.6428782943440924


In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='linear', decision_function_shape='ovr') # OVR stands for One-vs-Rest
clf.fit(X_train, y_train)

# Predict the test set labels
y_pred = clf.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [53]:
# b=[]
# for j in a:
#     b.append(tokenize_text(j[0]))

In [ ]:
a=create_word_set(tokens)
len(a)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA

# Example Persian text data
text_data = [
    'این یک متن نمونه است.',
    'این یک متن دیگر است.',
    'این هم یک متن دیگر است.',
    'این یک  یک متن آزمایشی دیگر است.'
]

# Convert text data to a matrix of word counts
vectorizer = CountVectorizer()
word_counts = vectorizer.fit_transform(text_data)

# Apply PCA to reduce the number of features
pca = PCA(n_components=2)
reduced_features = pca.fit_transform(word_counts.toarray())

# Print the reduced features
print(reduced_features)


[[ 1.00000000e+00 -5.47955771e-01]
 [-3.92523115e-17  2.40249667e-01]
 [ 1.17756934e-16  8.55661876e-01]
 [-1.00000000e+00 -5.47955771e-01]]


In [58]:
import math
from collections import Counter

def compute_tf(text):
    tf_text = Counter(text)
    for i in tf_text:
        tf_text[i] = tf_text[i]/float(len(text))
    return tf_text

def compute_idf(word, corpus):
    return math.log10(len(corpus)/sum([1.0 for i in corpus if word in i]))

def compute_tfidf(text, corpus):
    tfidf_text = {}
    tf_text = compute_tf(text)
    for word in tf_text:
        idf = compute_idf(word, corpus)
        tfidf_text[word] = tf_text[word] * idf
    return tfidf_text


In [67]:
import math
from hazm import word_tokenize

# Define a function to calculate the TF-IDF scores
def calculate_tfidf(texts):
    # Tokenize the texts
    tokens = [word_tokenize(text) for text in texts]

    # Count the number of documents that contain each token
    doc_freq = {}
    for doc in tokens:
        for token in set(doc):
            if token not in doc_freq:
                doc_freq[token] = 1
            else:
                doc_freq[token] += 1

    # Calculate the IDF scores for each token
    idf = {}
    num_docs = len(texts)
    for token, freq in doc_freq.items():
        idf[token] = math.log(num_docs / (freq + 1))

    # Calculate the TF-IDF scores for each token in each document
    tfidf_scores = []
    for doc in tokens:
        tf = {}
        for token in doc:
            if token not in tf:
                tf[token] = 1
            else:
                tf[token] += 1
        doc_tfidf = {}
        for token, freq in tf.items():
            if token in idf:
                doc_tfidf[token] = freq * idf[token]
        tfidf_scores.append(doc_tfidf)

    return tfidf_scores
